In [ ]:
import requests
import pandas as pd
import json
from bs4 import BeautifulSoup
from flask import Flask, request
from flask_restful import Api, Resource, reqparse

app = Flask(__name__)
api = Api(app)

class Admin(Resource): #objects of this class will be supplied for each request

    def post(self):
        
        data = request.get_json() #get data from request
        url = data['url']
        
        reqs = requests.get(url)
        soup = BeautifulSoup(reqs.text, 'lxml') #load the requested url to bs4

        headings=[]
        contents=[]
        temp = []
        count=0
        
        headings.append("Drug name is:") 
        contents.append(soup.find("h1").text) #extracting the drug name

        for heading in soup.find_all( ["h2","p"]): #loop will go through every h2 and p tags
            if(heading.name == "h2"): #this condition will executes if it finds a h2 tag
                count+=1
                if (count == 11): #after gettings the main headngs stop the loop
                    break
                headings.append(heading.text) #add the founded H2 heading to the list
                if(temp != []):
                    contents.append(temp) #add the previous founded p tags within the h2 tag to contents 
                temp = [] # reset the temp value to incluce new p tags

            if(heading.name == "p"): 
                temp.append(heading.text) #include the p tags founded to the content list

        contents.pop(1) #remove the founded description without heading
        headings.pop(10) # remove the heading founded after getting all the P tags

        df = pd.DataFrame({'subheadings':headings,'content':contents}) #add lists to the pandas data frame
        result = df.to_json(contents[0]+".json",orient="records") #print the content to a json file containing the medicine name
        result = df.to_json(orient="records")
        parsed = json.loads(result)
        json.dumps(parsed) #printing the results into jupiter view
        
        return parsed, 200 #returning the data to the relevent request with the request code 200

# Add URL endpoints
api.add_resource(Admin, '/url') #adding the url path to the server

if __name__ == '__main__':
    app.run()